In [94]:
import json
import re
from datetime import datetime, timedelta
import spacy
import en_core_web_sm
import csv
import json
import re
from spacy.matcher import Matcher

nlp = en_core_web_sm.load()


file_name = 'indeed_postings_details_1119.jsonl'
crawl_date = '2022' + re.search('[0-9]+', file_name).group(0)
crawl_date = datetime.strptime(crawl_date, '%Y%m%d')
output_file = open('indeed_postings_IE.jsonl', 'w')

with open(file_name, 'r') as input_file:
    json_list = list(input_file)

In [95]:
class IndeedJobPosting():
    def __init__(self, title, hiringOrganization, jobLocation, totalJobOpenings, employerOverview, responsibilities, qualifications, baseSalary, workHours, experienceRequirements):
        self.title = title
        self.hiringOrganization = hiringOrganization
        self.jobLocation = jobLocation
        self.totalJobOpenings = totalJobOpenings
        self.employerOverview = employerOverview
        self.responsibilities = responsibilities
        self.qualifications = qualifications
        self.baseSalary = baseSalary
        self.workHours = workHours
        self.experienceRequirements = experienceRequirements

In [96]:
pos_list = ['ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB']
nouns_list = ['proficiency', 'experience', 'familiarity', 'knowledge', 'expertise', 'understanding', 'ability', 'you']


duties_patterns = [
    [{'POS': 'SPACE'}, {'POS': 'VERB', 'IS_TITLE': True}, {'POS': {'IN': pos_list}, 'OP': '+'}]
]

qualifications_patterns = [
    [{'POS': 'SPACE'}, {'LOWER': {'IN': nouns_list}}, {'POS': {'IN': pos_list}, 'OP': '+'}]
]

salary_patterns = [
    [{'ORTH': '$'}, {'POS': 'NUM'}, {'POS': {'IN': pos_list}, 'OP': '+'}]
]

schedule_patterns = [
    [{'LOWER': 'schedule'}, {'ORTH': ':', 'OP': '?'}, {'POS': {'IN': pos_list}, 'OP': '+'}]
]

skills_patterns = [
    [{'POS': {'IN': pos_list}, 'OP': '+'}, {'ORTH': ':'}, {'POS': 'NUM'}, {'ORTH': {'IN': ['year', 'years']}}]
]


count = 0
for json_str in json_list:
    job = json.loads(json_str)
    datePosted = re.search('[0-9]', job['posting_date']) # might be employer reviewed date
    if datePosted:
        datePosted = crawl_date - timedelta(days=int(datePosted.group(0)))
    else:
        datePosted = crawl_date
    hiringOrganization = job['company']
    jobLocation = job['location']
    title = job['job_title']
    totalJobOpenings = job['num_candidate']
    employerOverview = job['job_description']
    

    responsibilities = []
    qualifications = []
    salary = None
    workHours = None
    experiences = []
    doc = nlp(job['job_description'])
#         for w in doc:
#             print(f'{w.text:15s} [{w.tag_:5s} | {w.pos_:6s} | {w.lemma_:6s}')
    matcher = Matcher(nlp.vocab) 
    matcher.add("duties", duties_patterns, greedy="LONGEST") 
    matches = matcher(doc)
    if matches:
        for match_id, start, end in matches:
            span = doc[start:end]
            responsibilities.append(span.text.replace('\n', ''))


    matcher = Matcher(nlp.vocab) 
    matcher.add("qualifications", qualifications_patterns, greedy="LONGEST") 
    matches = matcher(doc)
    if matches:
        for match_id, start, end in matches:
            span = doc[start:end]
            qualifications.append(span.text.replace('\n', ''))


    matcher = Matcher(nlp.vocab) 
    matcher.add("salary", salary_patterns, greedy="LONGEST") 
    matches = matcher(doc)
    if matches:
        for match_id, start, end in matches:
            span = doc[start:end]
            salary = span.text.replace('\n', '')
            
    
    matcher = Matcher(nlp.vocab) 
    matcher.add("schedule", schedule_patterns, greedy="LONGEST") 
    matches = matcher(doc)
    if matches:
        for match_id, start, end in matches:
            span = doc[start:end]
            if span.text.replace('\n', '').lower() != 'schedule:':
                workHours = span.text.replace('\n', '')
                
    matcher = Matcher(nlp.vocab) 
    matcher.add("skills", skills_patterns, greedy="LONGEST") 
    matches = matcher(doc)
    if matches:
        for match_id, start, end in matches:
            span = doc[start:end]
            experiences.append(span.text.replace('\n', ''))
    
    job_posting = IndeedJobPosting(title, hiringOrganization, jobLocation, totalJobOpenings, employerOverview, responsibilities, qualifications, salary, workHours, experiences)
    line = json.dumps(job_posting.__dict__) + "\n"
    output_file.write(line)
    count += 1
    
output_file.close()